In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch

from keras.layers import Input
from keras.models import Model

2024-04-29 17:44:11.744116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 17:44:12.221498: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BATCH_SIZE = 512

In [3]:
AS_dataset = pd.read_csv('./hairpin_filter_dataset.csv', encoding='utf-8')
full_X = AS_dataset.loc[:,'freq':'S1'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[:,'S21r':'S21i'].to_numpy(dtype = np.float32)

In [4]:
# 归一化
# full_X[0] = (full_X[0] - 0.1) / (9.0 - 0.1)
full_X[:, 1] = (full_X[:, 1] - 200) / (2000 - 200)
full_X[:, 2] = (full_X[:, 2] - 200) / (2000 - 200)
full_X[:, 3] = (full_X[:, 3] - 200) / (2000 - 200)
full_X[:, 4] = (full_X[:, 4] - 200) / (4500 - 200)
full_X[:, 5] = (full_X[:, 5] - 200) / (4500 - 200)
full_X[:, 6] = (full_X[:, 6] - 200) / (4500 - 200)
full_X[:, 7] = (full_X[:, 7] - 200) / (4500 - 200)
full_X[:, 8] = (full_X[:, 8] - 9000) / (12000 - 9000)
full_X[:, 9] = (full_X[:, 9] - 100) / (300 - 100)

In [5]:
X_train, X_vali, y_train, y_vali = train_test_split(full_X, full_y, test_size=0.05, random_state=0)

In [6]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-04-29 17:44:14.718742: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-29 17:44:14.719217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [7]:
dataset_vali = tf.data.Dataset.from_tensor_slices((X_vali, y_vali))
dataset_vali = dataset_vali.shuffle(buffer_size=X_vali.shape[0])
dataset_vali = dataset_vali.batch(BATCH_SIZE)
dataset_vali = dataset_vali.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
# 定义模型构建函数
def build_model(hp):
    model = tf.keras.Sequential()
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(
            tf.keras.layers.Dense(
                units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32), 
                activation='leaky_relu' if hp.Boolean('use_relu' + str(i)) else None
            )
        )
        if hp.Boolean('use_batchnorm' + str(i)):
            model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(2))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mse')
    return model

In [9]:
# 创建 tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=1024, # 根据实际情况调整试验次数
    directory='my_dir',
    project_name='mlp_project1')

In [ ]:
# 开始搜索过程
tuner.search(dataset_train,
             epochs=5,
            validation_data=dataset_vali)

Trial 18 Complete [00h 00m 41s]
val_loss: 0.06102102994918823

Best val_loss So Far: 0.0368754044175148
Total elapsed time: 00h 12m 40s

Search: Running Trial #19

Value             |Best Value So Far |Hyperparameter
5                 |5                 |num_layers
512               |672               |units_0
False             |False             |use_relu0
True              |True              |use_batchnorm0
0.0001            |0.001             |learning_rate
928               |416               |units_1
False             |True              |use_relu1
False             |True              |use_batchnorm1
224               |448               |units_2
False             |False             |use_relu2
True              |False             |use_batchnorm2
1024              |192               |units_3
False             |True              |use_relu3
False             |False             |use_batchnorm3
896               |608               |units_4
False             |True              |use_relu4
